In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



class Suggestions(BaseModel):
    words: List[str] = Field(description="""list of substitute words based on context""")
    ### Throws error in case of numbered list from API
    @validator('words')
    def not_start_with_number(cls, field):
        
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
        return field
    
parser = PydanticOutputParser(pydantic_object=Suggestions)

In [3]:
from langchain.prompts import PromptTemplate


template = """
Offer a list suggestions to substitute the specified target_word based the presented contenxt.
{format_instructions}
target_word={target_word}
context={context}
"""


target_word="behaviour"
context="""The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."""


prompt_template = PromptTemplate(
template=template,
input_variables=["target_word", "context"],
partial_variables={"format_instructions": parser.get_format_instructions() }
)

In [5]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"words": {"title": "Words", "description": "list of substitute words based on context", "type": "array", "items": {"type": "string"}}}, "required": ["words"]}
```


In [8]:
from langchain.chat_models import  ChatOpenAI
from langchain import LLMChain

model_name = "gpt-3.5-turbo"
temperature = 0.0
model = ChatOpenAI(model_name=model_name, temperature=temperature)

chain = LLMChain(llm=model, prompt=prompt_template)


output = chain.run({"target_word": target_word, "context": context})

parser.parse(output)

Suggestions(words=['conduct', 'actions', 'demeanor', 'attitude'])

## Multiple Output Example

In [9]:
template = """
Offer a list of suggestions to substitute the specified target_word based on the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""


class Suggestions(BaseModel):
    words: List[str] = Field(description="""list of substitute words based on context""")
    reasons: List[str] = Field(description = """The reasoning of why this word fits the context""")
        
    @validator('words')
    def not_start_with_number(cls, field):
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
                
        return field
    
    
    
    @validator("reasons")
    def end_with_dot(cls, field):
        for idx, item in enumerate(field):
            if item[-1] != ".":
                field[idx] += "."
        return field
            

In [10]:
Suggestions(words=['conduct', 'manner', 'demeanor', 'comportment'], reasons=['refers to the way someone acts in a particular situation.', 
                                                                             'refers to the way someone behaves in a particular situation.',
                                                                             "refers to the way someone behaves in a particular situation.", 
                                                                            "refers to the way someone behaves in a particular situation."])





Suggestions(words=['conduct', 'manner', 'demeanor', 'comportment'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.', 'refers to the way someone behaves in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

In [11]:


parser = PydanticOutputParser(pydantic_object=Suggestions)
prompt_template = PromptTemplate(
template=template,
input_variables=["target_word", "context"],
partial_variables={"format_instructions": parser.get_format_instructions() }
)

In [13]:
chain = LLMChain(llm=model, prompt=prompt_template)


output = chain.run({"target_word": target_word, "context": context})

parser.parse(output)

Suggestions(words=['conduct', 'manage', 'handle', 'facilitate'], reasons=['These words all imply the act of guiding or overseeing a situation, which is what the teacher was trying to do in the classroom context.'])

In [14]:
output

'{\n  "words": ["conduct", "manage", "handle", "facilitate"],\n  "reasons": ["These words all imply the act of guiding or overseeing a situation, which is what the teacher was trying to do in the classroom context."]\n}'

**CommaSeparatedOutputParser**

In [15]:
from langchain.output_parsers import CommaSeparatedListOutputParser


parser = CommaSeparatedListOutputParser()

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


template = """
offer a list of suggestions to substitute the word '{target_word}' based on the presented following text: {context}.
{format_instructions}
"""


prompt_template= PromptTemplate(
template=template,
input_variables=["target_word", "context"],
partial_variables={"format_instructions": parser.get_format_instructions()}
)


chain = LLMChain(llm=model, prompt=prompt_template)

# Run the LLMChain to get the AI-generated answer


output = chain.run({"target_word": target_word, "context":context})

parser.parse(output)

['conduct', 'demeanor', 'actions', 'conduct', 'mannerisms']

In [19]:
print(output)

conduct, demeanor, actions, conduct, mannerisms


**StructuredOutputParser**

In [32]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


response_schemas = [
    ResponseSchema(name="words", description="A substitute word based on context."),
    ResponseSchema(name="reasons", description = """The reasoning of why this word fits the context.""")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [33]:
parser.get_format_instructions()

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"words": string  // A substitute word based on context.\n\t"reasons": string  // The reasoning of why this word fits the context.\n}\n```'

the above class offers no particular advantage compared to pydanticOutputClass as it offers validation and enhanced flexibility for more intricate tasks

In [34]:
prompt_template= PromptTemplate(
template=template,
input_variables=["target_word", "context"],
partial_variables={"format_instructions": parser.get_format_instructions()}
)


chain = LLMChain(llm=model, prompt=prompt_template)

output = chain.run({"target_word": target_word, "context":context})
output

'```json\n{\n\t"words": "conduct",\n\t"reasons": "The disruptive behaviour of the students hindered the teacher\'s ability to conduct the lesson effectively."\n}\n{\n\t"words": "demeanor",\n\t"reasons": "The demeanor of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."\n}\n{\n\t"words": "attitude",\n\t"reasons": "The attitude of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."\n}\n{\n\t"words": "actions",\n\t"reasons": "The actions of the students in the classroom were disruptive and made it difficult for the teacher to conduct the lesson."\n}\n{\n\t"words": "conduct",\n\t"reasons": "The conduct of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."\n}\n```'

### Fixing Errors

In [36]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


class Suggestions(BaseModel):
    words: List[str] = Field(description = """List of substitute words based on context""")
    reasons: List[str] = Field(description = """the reasonsing of why this word fits the context""")
        
        
parser = PydanticOutputParser(pydantic_object=Suggestions)



missformatted_output = '{"words": ["conduct", "manner"], "reasoning":["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}'
    
parser.parse(missformatted_output)

OutputParserException: Failed to parse Suggestions from completion {"words": ["conduct", "manner"], "reasoning":["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}. Got: 1 validation error for Suggestions
reasons
  field required (type=value_error.missing)

### Correcting misformatted output

In [37]:
from langchain.output_parsers import OutputFixingParser

outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

In [38]:
missformatted_output = '{"words":["conduct", "manner"]}'
outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=['These words accurately describe behavior in a specific situation.', 'They are synonyms that convey the intended meaning effectively.'])

### Retry Output Parser

In [41]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


class Suggestions(BaseModel):
    words: List[str] = Field(description = """list of substitute words based on context""")
    reasons: List[str] = Field(description="""the reasoning of why this word fits the context""")
        
        
        
parser = PydanticOutputParser(pydantic_object=Suggestions)



template = """
Offer a list of suggestions to substitute the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""


prompt = PromptTemplate(
template=template,
input_variables=["target_word", "context"],
partial_variables = {"format_instructions": parser.get_format_instructions()}
)



model_input = prompt.format_prompt(target_word="behaviour", 
                                   context="""The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.""")




In [42]:
from langchain.output_parsers import RetryWithErrorOutputParser

missformatted_output = '{"words":["conduct", "manner"]}'

retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)

retry_parser.parse_with_prompt(missformatted_output, model_input)





Suggestions(words=['conduct', 'manner'], reasons=["The word 'conduct' can be used as a substitute for 'behaviour' in this context as it refers to the way in which something is carried out or managed.", "The word 'manner' can also be used as a substitute for 'behaviour' as it pertains to the way in which something is done or happens."])